In [1]:
cd ..

C:\Users\rajag\Desktop\convolve\convolve_epoch_1


In [2]:
cd data

C:\Users\rajag\Desktop\convolve\convolve_epoch_1\data


In [3]:
# import libraries
import pandas as pd 
import numpy as np
import json

In [4]:
# custom settings 
pd.set_option('display.max_colwidth', None)

In [5]:
# get the data 
with open('train.json', 'r') as f:
    dict_train = json.load(f)

df = pd.DataFrame.from_dict(dict_train, orient='index')
df.reset_index(level=0, inplace=True)
df.rename(columns = {'index':'log',0:'status'}, inplace = True)

In [6]:
# Prepare the training data
df_parsed = df.copy(deep=True)
df_parsed["log"]= df_parsed["log"].str.split(":", n = 3, expand = False)
df_temp = df_parsed.copy(deep=True)
df_temp = pd.DataFrame(df_parsed['log'].to_list(), columns=['log1','log2','log3','log4'])
df_temp['status'] = df_parsed['status']
df4 = df_temp[df_temp['log4'].isnull()].copy(deep=True)
df5 = df_temp[df_temp['log4'].notnull()].copy(deep=True)
train1 = df4.drop(['log4', 'log1', 'log2'], axis=1)
train2 = df5.drop(['log3', 'log1', 'log2'], axis=1)
train1.rename(columns = {'log3':'log'}, inplace = True)
train2.rename(columns = {'log4':'log'}, inplace = True)
tmp_list = [train1,train2]
train = pd.concat(tmp_list)
train = train.dropna()

# Trying to deal with the imbalance in the dataset 

In [7]:
# method 1
# doing a 50 50 split

train_new1 = train.sample(frac=1)

fraud_df = train_new1.loc[train_new1['status'] == "abnormal"]
non_fraud_df = train_new1.loc[train_new1['status'] == "normal"][:69692]

normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

# Shuffle dataframe rows
balanced_train1 = normal_distributed_df.sample(frac=1, random_state=42)

In [8]:
# method 5, custom sampler
total_normal = train[train["status"] == "normal"]
total_abnormal = train[train["status"] == "abnormal"]

sampled_total_normal = total_normal.sample(n=69692)

normal_distributed_df = pd.concat([total_abnormal, sampled_total_normal])

balanced_train2 = normal_distributed_df.sample(frac=1,random_state=42)

In [9]:
# method 2
# Over sampling- making copies of the minority class

X = train.drop('status',axis=1)
y = train['status']
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
#split data into test and training sets
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
#combine them back for resampling
train_data = pd.concat([X_train, y_train], axis=1)
# separate minority and majority classes
negative = train_data[train_data.status=="abnormal"]
positive = train_data[train_data.status=="normal"]
# upsample minority
pos_upsampled = resample(positive,
 replace=True, # sample with replacement
 n_samples=len(negative), # match number in majority class
 random_state=27) # reproducible results
# combine majority and upsampled minority
upsampled = pd.concat([negative, pos_upsampled])
# check new class counts
upsampled.status.value_counts()

abnormal    46892
normal      46892
Name: status, dtype: int64

In [10]:
# method 3
# downsample majority

neg_downsampled = resample(negative,
 replace=True, # sample with replacement
 n_samples=len(positive), # match number in minority class
 random_state=27) # reproducible results
# combine minority and downsampled majority
downsampled = pd.concat([positive, neg_downsampled])
# check new class counts
downsampled.status.value_counts()

abnormal    2676259
normal      2676259
Name: status, dtype: int64

In [14]:
# method 4
# over sampling using smote 
# data first needs to be vectorized to use smote
from imblearn.over_sampling import SMOTE
# Separate input features and target
X = df.drop('status',axis=1)
y = df['status']
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)
sm = SMOTE(random_state=27)
X_train, y_train = sm.fit_resample(X_train, y_train)
y_train = pd.DataFrame(y_train, columns = ['status'])
balanced_train3 = pd.concat([X_train, y_train], axis=1)

ValueError: could not convert string to float: ' 1121309185 2005.07.13 R04-M1-NF-C:J08-U01 2005-07-13-19.46.25.912987 R04-M1-NF-C:J08-U01 RAS KERNEL INFO generating core.7234\n'

# Tokenizer 
### Tokenizer is used to split sentences to words or chracters etc
###### (Specific libraries do their own shit tho)

In [ ]:
# tokenizer 1

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(new_df['log3'].values)
X = tokenizer.texts_to_sequences(new_df['log3'].values)
X = pad_sequences(X)

Y = pd.get_dummies(new_df['status']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
# Tokenizer 2
# reference -> https://www.kaggle.com/code/ashokkumarpalivela/sentiment-analysis-with-machine-learning

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

tfidf_vectorizer = TfidfVectorizer(max_features=10000)
tfidf_vectorizer.fit(X_train)

# transform
tfidf_X_train = tfidf_vectorizer.transform(X_train)
tfidf_X_test = tfidf_vectorizer.transform(X_test)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=10000)
tfidf_balanced_dataset = tfidf_vectorizer.fit_transform(balanced_train2)

# Vectorizer
### Vectorizer converts words to integers

# Model

In [ ]:
# model 1 used with tokenizer 1
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size, verbose = 1)

In [ ]:
Y_pred = model.predict(X_test,batch_size = batch_size)

In [ ]:
# reference for the above shjit -> https://www.kaggle.com/code/sanjay11100/lstm-sentiment-analysis-data-imbalance-keras/notebook

In [ ]:
# model 2



In [ ]:
# model 3
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb

In [ ]:
model3 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [ ]:
# tokenization done using tokenizer 1 
train_model3 = model3.fit(X_train, Y_train)
pred3 = train_model3.predict(X_test)

In [ ]:
print(classification_report(Y_test,pred3))